In [13]:
import numpy as np

In [14]:
#   .txt
file_name = 'academicMisconduct.txt'

### Deliverable 1


In [15]:
file = open(file_name, 'r')
content = file.read()
file.close()

num_char = len(content)

unique_char = set(content)
num_unique_char = len(unique_char)

# each character is encoded by the same number of bits
average_bit = np.round((np.log(num_unique_char)), 2)
total_bit = np.round(average_bit * num_char)

print('Total number of characters: {} \nNumber of distinct characters: {} \nTotal number of bits needed to encode the text: {} \nAverage number of bits per character: {}'.format(
    num_char, num_unique_char, total_bit, average_bit))

Total number of characters: 9881 
Number of distinct characters: 65 
Total number of bits needed to encode the text: 41204.0 
Average number of bits per character: 4.17


### Deliverable 2


In [16]:
file = open(file_name, 'r')
content = file.read()
file.close()

num_char = len(content)

char_freq = dict()
for char in content:
    char_freq[char] = char_freq.get(char, 0) + 1

# sort unique characters by frequency in descending order
sorted_unique_chars = sorted(
    char_freq.keys(), key=lambda char: char_freq[char], reverse=True)

total_bit = 0
i = 1
for char in sorted_unique_chars:
    total_bit += char_freq[char] * i
    i += 1

average_bit = np.round(total_bit / num_char, 2)

print('Total number of bits: {} \nAverage number of bits: {}'.format(
    total_bit, average_bit))

Total numer of bits: 85373 
Average number of bits: 8.64


### Deliverable 3


In [17]:
class Node:
    '''
    Represents a node in the binary tree of unique characters.

    Attributes:
        chars (list): a list of unique characters.
        freq (int): total frequency of the characters present in `chars`.
        left (Node): the left-child node.
        right (Node): the right-child node.
    '''

    def __init__(self, chars, freq, left=None, right=None):

        self.chars = chars
        self.freq = freq
        self.left = left
        self.right = right

In [18]:
def get_char_freq(file_name):
    '''
    Returns a dictionary of unique characters and their corresponding frequency in a text file.

        Parameter:
            file_name (str): the name of a .txt file.

        Return:
            char_freq (dict): a dictionary of unique characters and their corresponding frequency in a text file.
    '''
    file = open(file_name, 'r')
    content = file.read()
    file.close()

    char_freq = dict()
    for char in content:
        char_freq[char] = char_freq.get(char, 0) + 1

    return char_freq

In [19]:
def create_node(char_freq):
    '''
    Returns a list of nodes.

        Parameter:
            char_freq (dict): a dictionary of unique characters and their corresponding frequency in a text file.

        Return:
            nodes (list): a list of Node objects.
    '''
    nodes = list()

    for char, freq in char_freq.items():
        nodes.append(Node([char], freq))

    return nodes

In [20]:
def merge_trees(left_root, right_root):
    '''
    Merging two Node objects.

        Parameters:
            left_node (Node): a node to be the left-child node.
            right_node (Node): a node to be the right_child node.

        Return:
            A Node object that is the parent node of the input nodes.f
    '''
    chars = list(set(left_root.chars + right_root.chars))
    freq = left_root.freq + right_root.freq

    return Node(chars, freq, left_root, right_root)

In [21]:
def create_binary_tree(nodes):
    '''
    Create a binary tree from a list of nodes.

        Parameter:
            nodes (list): a list of Node objects.

        Return:
            root (Node): the root of the binary tree that is created from the input list of nodes.
    '''
    sorted_nodes = sorted(nodes, key=lambda x: x.freq)

    root = sorted_nodes[0]
    for i in range(1, len(sorted_nodes), 1):
        root = merge_trees(sorted_nodes[i], root)

    return root

In [22]:
test_case = {'a': 20, 'b': 2, 'c': 1}
root = create_binary_tree(create_node(test_case))

### Deliverable 4


In [23]:
def find_in_tree(char, root):
    '''
    Searches in a tree and returns the level of the node containing a character, or None if it is not found.

        Parameters:
            char (str): a character.
            root (Node): a root node of a tree.

        Return:
            level (int): the level of the node contating the character, None if otherwise.
    '''
    # BFS
    level = 0
    nodes = [root]

    while nodes:
        next_nodes = list()

        for node in nodes:
            if len(node.chars) == 1 and node.chars[0] == char:
                return level

            if node.left:
                next_nodes.append(node.left)

            if node.right:
                next_nodes.append(node.right)

        nodes = next_nodes
        next_nodes = list()
        level += 1

    return None

In [24]:
find_in_tree('a', root)

1

In [25]:
find_in_tree('b', root)

2

In [26]:
find_in_tree('c', root)

2

In [27]:
find_in_tree('d', root)

### Deliverable 5


In [28]:
def tree_from_file(file_name):
    '''
    Prints the total number of characters, the number of distinct characters, the total number of bits needed to encode the text, and the average number of bits per characcter.

        Parameter:
            file_name (str): the name of a .txt file.
    '''
    file = open(file_name, 'r')
    content = file.read()
    file.close()

    char_freq = dict()
    for char in content:
        char_freq[char] = char_freq.get(char, 0) + 1

    nodes = list()
    for char, freq in char_freq.items():
        nodes.append(Node([char], freq))

    while len(nodes) > 1:
        right_node = min(nodes, key=lambda node: node.freq)
        nodes.remove(right_node)

        left_node = min(nodes, key=lambda node: node.freq)
        nodes.remove(left_node)

        new_node = merge_trees(left_node, right_node)
        nodes.append(new_node)

    root = nodes[0]

    def calculate_bit(node, level=0):
        if node.left is None and node.right is None:
            return node.freq * level
        else:
            level += 1
            return calculate_bit(node.left, level) + calculate_bit(node.right, level)

    total_bit = calculate_bit(root)

    num_char = len(content)
    average_bit = np.round(total_bit / num_char, 2)

    print('Total number of characters: {} \nNumber of distinct characters: {} \nTotal number of bits needed to encode the text: {} \nAverage number of bits per character: {}'.format(
        num_char, len(sorted_unique_chars), total_bit, average_bit))

In [29]:
tree_from_file(file_name)

Total number of characters: 9881 
Number of distinct characters: 65 
Total number of bits needed to encode the text: 44218 
Average number of bits per character: 4.48
